An implementation of encoded $\ket{A^2} = \ket{\sqrt{-iY}} = e^{-i \frac{\pi}{4}Y} \ket{+} = \ket{1}$ preparation in the Steane code using stim.

In particular, $A^2 = XH$, which will be used to implement the gate in stim.

Maybe easiest to just do $Y$ eigenstate, $\ket{+i}$?

In [1]:
from stim import Circuit
from noisy_circuits import noisy_steane_encoder,noisy_encoded_y
from decoders import steane_transversal_decoder

The logical $X$ outcome for the repetition-encoded ancilla block indicates which eigenstate the Hadamard test tells us it projected onto.

The logical $Y$ outcome for the Steane-encoded data block will indicate which eigenstate is observed.

If these two observables do not match, then there is a logical error on the output.

Below is a test of the circuit without errors, which should give us no errors and always result in selected trials.

In [2]:
def construct_noisy_circuit_steane2(perr:float) ->Circuit:
    r"""
    Method for producing a noisy circuit that in the absence of errors
    will initialize and measure the logical Pauli-Y eigenstate |+i>.
    In this particular circuit, the both the logical qubit and ancilla are
    encoded into the Steane code.
    """
        
    noisy_circ = Circuit()

    steaneBlockA = [0,1,2,3,4,5,6]
    steaneBlockB = [7,8,9,10,11,12,13]

    noisy_circ.append("RX",steaneBlockA[0])
    noisy_circ.append("S_DAG",steaneBlockA[0]) # prep initial state to |+i>
    
    noisy_circ.append("RX",steaneBlockB[0]) # prep unencoded ancilla in |+>

    noisy_circ.append("DEPOLARIZE1",[steaneBlockA[0],steaneBlockB[0]],perr) # apply noise for qubit initialization

    noisy_circ = noisy_steane_encoder(noisy_circ,steaneBlockA,perr)
    noisy_circ = noisy_steane_encoder(noisy_circ,steaneBlockB,perr)

    noisy_circ = noisy_encoded_y(noisy_circ,steaneBlockA,steaneBlockB,perr)

    noisy_circ.append("DEPOLARIZE1",steaneBlockB,perr)
    noisy_circ.append("MX",steaneBlockB) # noisy readout transversal logical-X
    noisy_circ.append("MY",steaneBlockA) # ideal readout transversal logical-Y
        
    return noisy_circ

In [3]:
def decode_steane2_output(rec:list[bool]) -> tuple[bool,bool]:

    xs = rec[:7]
    ys = rec[7:]
    
    mx = steane_transversal_decoder(xs)
    my = steane_transversal_decoder(ys)
 
    return mx,my

In [4]:
error_rates = [0,
              1.0*10**(-5), 2.5*10**(-5),5.0*10**(-5),7.5*10**(-5),
              1.0*10**(-4), 2.5*10**(-4),5.0*10**(-4),7.5*10**(-4),
              1.0*10**(-3), 2.5*10**(-3),5.0*10**(-3),7.5*10**(-3),
              1.0*10**(-2)]

results = []

shots = 10_000_000

for perr in error_rates:
    selected = 0
    errors = 0

    noisy_circ = Circuit()
    
    noisy_circ = construct_noisy_circuit_steane2(perr)
    sampler = noisy_circ.compile_sampler()
    sample = sampler.sample(shots)

    for rec in sample:
        mx,my = decode_steane2_output(rec)
        if not mx: # initialized with S_DAG
            selected +=1
            errors += not (mx==my)
        
    print(str(perr) + ', ' + str([selected/shots,errors/selected]))
    print()

0, [1.0, 0.0]

1e-05, [0.9998806, 0.0]

2.5e-05, [0.9997093, 0.0]

5e-05, [0.9994097, 9.005315837939135e-07]

7.500000000000001e-05, [0.9991151, 1.000885683741543e-06]

0.0001, [0.9988209, 3.0035414757540617e-06]

0.00025, [0.997019, 1.7853220450161933e-05]

0.0005, [0.9939038, 7.254223195444066e-05]

0.00075, [0.9907697, 0.0001631055128149357]

0.001, [0.9875304, 0.0002768522366501325]

0.0025, [0.9666, 0.0017257397061866335]

0.005, [0.9286267, 0.00673241465058026]

0.0075, [0.888523, 0.014613465267640792]

0.01, [0.8489557, 0.02520885365396569]

